# Functions, Mutability, and Solving (basic) ODE

## Functions

So far, we have learned about how to store data/values into variables within Python. In order for this to be mathematically useful, we also need to be able to define functions which act on this data.

In Python, functions are created using the `def` keyword, followed by the name of the function and its arguments within parentheses. And then a colon. Like this:

In [2]:
import math # Importing the math module to use the exp function that it contains

def std_logistic(x):
    """Calculates the standard logistic function.

    A longer description of the function can be added here if needed.

    Args:
        x (float): The input value.

    Returns:
        float: The output of the standard logistic function.
    """
    return 1 / (1 + math.exp(-x))

# If you have run this cell, you can then call the std_logistic function with any value of x to get the output of the standard logistic function. For example:
print(std_logistic(0))  # Output: 0.5

0.5


Let's discuss the pieces of this. First, the function name.
- It is good practice to use only lowercase letters for the name of functions and variables.
- The name should be descriptive and not something like "f". Underscores can be used as spaces.
- Inside the parentheses, you list (separated by commas) the variables the function should take in.
- These variables exist ONLY inside the function. That means:
    - If there is an `x` variable outside the function, this `x` is different from that one and they won't interact in any way. (**But** the data it contains MAY NOT be different. More on this below.)
    - You cannot access the variable `x` outside of this function.

Best practice is to include a multi-line comment at the top of each function using triple quotations (single or double). This is called the "docstring". Help functions and intellisense can access it, which means that VS Code can remind you how to use your own function if you write it well!
- The first line of the docstring is a brief discriptor of what the function does.
- You can add a longer description under that as-needed.
- There should be a section called "Args" or "Arguments" or "Parameters" which lists (in order) what the function expects to take in.
- Good practice is to include the type of variable each argument expects. Here, a floating point number, which means any decimal (including integers)
- If the function returns some data (not all do), there should be a section called "Returns" which explains what the function will return. This is so that the user can expect that behavior and store any needed values being returned in a variable.

Finally, if the function is going to return a value, this is done with the keyword `return`. Multiple values can be returned, separated by a comma. This effectively returns a tuple of values.

In [4]:
# You can access a function as many times as you want, and you can compose functions.
print(std_logistic(std_logistic(0)))
some_number = std_logistic(0.5) + std_logistic(1)
print(some_number)

0.6224593312018546
1.3535179098318595


In mathematics, we often do a type of programming called *Functional Programming*. This is simply a paradigm we adopt to help us understand how our code works. The rules (which Python does not enforce) are:
1. We pass data to a function only through its arguments.
    - In Python, if you reference a variable name inside a function, and that variable name hasn't been defined inside the function and wasn't one of its arguments, Python will go looking for that variable outside of the function.
    - From the standpoint of functional programming, this is bad because it is disorderly and can introduce hard to diagnose errors as variables change. You want to think of a function as a black box with clear ins and outs: it takes data in through its parentheses and spits data out through its return. Everything else is internal to that function only.

In [5]:
# This is an example of what not to do in functional programming (but it runs without error):
K = 100
def dxdt(x):
    """Calculates the derivative of x with respect to time using the logistic growth model."""
    return x * (1 - x / K)
print(dxdt(50))  # Output: 25.0
# Instead, K should be defined within the function or passed as an argument to avoid side effects and improve code readability.

25.0


2. Return the needed result of a calculation with a return statement (as above).
    - While this seems obvious, there are other ways of returning values related to rule #3.

3. Do not modify the arguments of a function within the function.
    - E.g., if I pass a list to a function, I can use that list in my computations, but I should not assign to or alter the list in any way.
    - This is because **the list will actually change outside the function too.** However, that is not true for all data types.

An example is below, which will then bring us to our next topic: mutability. This will explain why this happens to some data types and not others.

In [7]:
import numpy as np

# Let's suppose we are computing eigenvectors for a given 2x2 matrix and given eigenvalue.
# Note: In practice, you would use a library function like numpy.linalg.eig to compute the eigenvectors.

def check_eigenvector(matrix2x2, eigenvalue):
    """Checks if the given eigenvalue is indeed an eigenvalue of the matrix.

    Args:
        matrix2x2 (numpy.ndarray): The input matrix, a 2x2

    Returns:
        bool: True if the eigenvalue is an eigenvalue of the matrix, False otherwise.
    """

    # Calculate A - λI, where A is the input matrix and λ is the eigenvalue
    matrix2x2[0,0] -= eigenvalue # short-hand for matrix2x2[0,0] = matrix2x2[0,0] - eigenvalue
    matrix2x2[1,1] -= eigenvalue

    # Calculate the determinant of A - λI and check if it is zero
    determinant = matrix2x2[0,0] * matrix2x2[1,1] - matrix2x2[0,1] * matrix2x2[1,0]
    return determinant == 0

# Now lets see what happens when we call this function with a specific matrix and eigenvalue:
A = np.array([[4, 2], [0, 3]]) # Define a 2x2 matrix (upper triangular)
eigenvalue = 4 # Define an eigenvalue
is_eigenvalue = check_eigenvector(A, eigenvalue) # Check if it is an eigenvalue
print(is_eigenvalue) # Output: True or False

True


Great! The above code returns True. Now, what was the value of our matrix A again...?

In [8]:
print(A)

[[ 0  2]
 [ 0 -1]]


Uh oh!!! What happened? An error like this could be disasterous in your code. And since no exception or warning was thrown, you've got no idea where to begin looking for the problem.

This occured because Rule #3 was broken: we modified matrix2x2, the argument, inside the function. But it gets worse. The problem persists if we do this:

In [9]:
import numpy as np

def check_eigenvector_v2(matrix2x2, eigenvalue):
    """Checks if the given eigenvalue is indeed an eigenvalue of the matrix.

    Args:
        matrix2x2 (numpy.ndarray): The input matrix, a 2x2

    Returns:
        bool: True if the eigenvalue is an eigenvalue of the matrix, False otherwise.
    """

    # Create a copy of the input matrix to avoid modifying the original matrix
    newmatrix = matrix2x2 # To fix the issue, replace this line with: newmatrix = matrix2x2.copy(), or newmatrix = np.array(matrix2x2)

    # Calculate A - λI, where A is the input matrix and λ is the eigenvalue
    newmatrix[0,0] -= eigenvalue # short-hand for matrix2x2[0,0] = matrix2x2[0,0] - eigenvalue
    newmatrix[1,1] -= eigenvalue

    # Calculate the determinant of A - λI and check if it is zero
    determinant = newmatrix[0,0] * newmatrix[1,1] - newmatrix[0,1] * newmatrix[1,0]
    return determinant == 0

# Now lets see what happens when we call this function with a specific matrix and eigenvalue:
A = np.array([[4, 2], [0, 3]]) # Define a 2x2 matrix (upper triangular)
eigenvalue = 4 # Define an eigenvalue
print(check_eigenvector_v2(A, eigenvalue)) # Output: True or False
print(A) # Output: [[4 2] [0 3]] (the original matrix should remain unchanged) Nope!

True
[[ 0  2]
 [ 0 -1]]


Above, we created a copy, but only a "shallow" copy. In essence, newmatrix just points to matrix2x2 and its data. It didn't actually copy the data over. This is because numpy arrays are *mutable*.

## Key facts about mutable/immutable objects (assignment)

- If you use assignment (=) to make a copy of an *immutable* object, the assignment will do what you expect because the underlying data cannot be changed. The data is copied over to the new variable, and you have two completely separate copies.
- If you use assignment (=) to make a copy of a *mutable* object, **only a shallow copy is made**. That means the copy is merely a reference to the same data contained in the original (it is essentially just a link to that data). 
    - **The reason for this** is that it makes it cheap to pass the data around to functions that might need it. It's much faster and uses less memory to just pass a pointer to data that already exists than to copy it all and then pass that (a "deep copy").
    - But it means that if you change the data with one of the variables, you have also changed it for the other one.

In [10]:
my_list = [1,2,3] # lists are mutable
my_list_ref = my_list # both my_list and my_list_ref point to the same object in memory
my_list[0] = 0 # so when I alter my_list, my_list_ref is altered too!
print('my_list = {}'.format(my_list))
print('my_list_ref = {}'.format(my_list_ref))

my_list = [0, 2, 3]
my_list_ref = [0, 2, 3]


If you don't want this behavior, you can force a deep copy as follows:

In [12]:
my_list = [1,2,3]
my_list_cpy = list(my_list) # calling the creation function makes a deep copy
my_list[0] = 0 # so when I alter my_list, my_list_cpy remains the same
print('my_list = {}'.format(my_list))
print('my_list_cpy = {}'.format(my_list_cpy))

my_list = [0, 2, 3]
my_list_cpy = [1, 2, 3]


Note that you don't have to worry about this when you are just re-assigning the original variable because you are essentially just re-assigning/re-purposing a label. 
If another label (variable) still points to the data, it isn't erased from memory.

In [13]:
my_list = [1,2,3] # mutable
my_list_ref = my_list # this now refers to the same object in memory as my_list
my_list = [4,5,6] # a new object is created, and my_list now refers to it instead.
                  # but my_list_ref still refers to [1,2,3]
print('my_list = {}'.format(my_list))
print('my_list_ref = {}'.format(my_list_ref))

my_list = [4, 5, 6]
my_list_ref = [1, 2, 3]


As a side note, this is how memory is freed in Python for other uses: Any data that becomes orphened/unreachable because the last variable pointing was removed or reassigned is marked for deletion. Then, at some unannounced point when the program isn't otherwise too busy, the garbage collector will act and free up the memory. This is called automatic *garbage collection*.

Not sure if you have a shallow copy or a deep copy? You can test for this using the keyword "is". 
In Python, there are two different types of equivalency tests. 
- "==" tests for elementwise data equivalency. 
- "is" tests to see if they are actually the same object in memory. 

**Note** that with immutable objects, these two comparisons will always correspond because an immutable type is considered fundamentally itself. That is, the number 2 is the number 2 everywhere. There aren't copies of the number 2, there exists only one immutable 2 and many variables can utilize it. This remains true with more complicated immutable types, like tuples.

In [15]:
my_list = [1,2,3]
my_list_cpy = my_list
my_list2 = list(my_list)
print(my_list == my_list_cpy)
print(my_list is my_list_cpy)
print(my_list == my_list2)
print(my_list is my_list2)

True
True
True
False


In [16]:
my_tuple = (1,2,3) # tuples are an immutable collection of objects
my_tuple_cpy = my_tuple # they point to the same unchangable object that is (1,2,3),
                        #   just as if you had made two variables equal to the number 2.
print(my_tuple == my_tuple_cpy)
print(my_tuple is my_tuple_cpy)

True
True


For built-in Python types like lists and tuples, == returns True if all elements are the same and False otherwise. **In numpy**, you instead get a boolean array with the elementwise results. 

So, if you want to see if they are all equivalent, you have to call the .all() method on the resulting array. There is also a .any() method that will check to see if *any* of the data is elementwise the same.

In [20]:
import numpy as np # unnecessary if you already ran a cell above that imported it.
A = np.array([1,2,3])
B = A
C = A.copy()
print(A == B)
print((A==B).all())
print(A is B)
print(A is C)

[ True  True  True]
True
True
False


## Practice!

In the code cell below, explore this behavior with dictionaries. The function to create/copy a dictonary is `dict()`.

In the code cell below, create a function called `logistic_ode` that represents the right-hand side of the equation
$$\frac{dN}{dt} = rN\left(1 - \frac{N}{K}\right).$$
That is, it should take in one value (a float, `N`) and return the value of the derivative $dN/dt$. $r$ and $K$ are parameters, so you will need to set them *inside* the function. (Later, we will talk about ways to set parameters outside the function without breaking Functional Programming rule #1).

Let $K = 1000$ and $r=1.5$.

Do you need to worry about mutability here? Test your function by passing in and printing values where you know what the answer should be. E.g., $N=1000$.

Now change your function slightly so that it takes in two arguments, `t` (for time) and then `N` (for population) in that order. And then re-run the cell.

This is necessary when you want to use `solve_ivp` to numerically solve the equation (discussed below) because generally speaking, first-order ODEs can be functions of both time and the dependent variable. It just happens that the logistic differential equation is autonomous, meaning it has no explicit dependence on time.

## Solving and plotting an ODE

In the cell above, you defined a function that represent an ordinary differential equation (ODE). This is a standard differential equation in mathematical biology representing a population that grows *logistically* with effective growth rate $r$ and carrying capacity $K$. Solutions (for $r,K>0$ and initial conditions strictly between 0 and $K$) look like sigmoid curves.

It's a non-stiff ODE and once we supply an initial condition, it can easily be solved numerically using standard methods like Runge-Kutta 4(5) - also known as RK45, which is what ode45 in MATLAB uses. SciPy has this (and other) methods for solving ODE implemented as functions, so we can make use of the SciPy library to solve rather than writing a solver from scratch. This is true for many common numerical methods that are generally applicable - always see if there is something you can use in SciPy before trying to write your own!

Specifically, we will make use of the `solve_ivp` function located in the `scipy.integrate` module. It's documentation can be found [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html).

We'll also plot the solution to the ODE!

The main plotting library in Python is [matplotlib](https://matplotlib.org/), which is in the same ecosystem as NumPy, SciPy, and Jupyter. If you are coming from MATLAB, you're in luck - the syntax for matplotlib was directly inspired by MATLAB.

Matplotlib is a giant library capable of all kinds of visualizations, including interactive plots and animation/video. There is a complex backend that deals with things like color and drawing artists, and making matplotlib work on different operating systems and in different contexts (like embedded in a Jupyter notebook). The part of matplotlib that gives you the actual plotting functions is called **pyplot**.

So, for most applications, you just need to import pyplot from matplotlib.

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt # Just like numpy is conventionally imported as np, pyplot is conventionally imported as plt.

# Solve the ODE with given initial condition, start time, end time, and parameters.

N0 = np.array([10]) # initial condition for the ODE must be given as a numpy array

# We will solve the ODE from time tstart to time tstop.
tstart = 0
tstop = 10

# You can give the t_eval argument of solve_ivp a mesh of time points in order to
#   force solutions to be recorded at these points. This is convenient for plotting, 
#   and it also allows you to compare solutions across different parameter values at the same time points.
# If you do not specify this, the solver will choose.

tmesh = np.linspace(tstart,tstop,1000) # This creates a numpy array of 1000 equally spaced points between tstart and tstop, inclusive.

### call the solver ###
# The default solver is RK45 which is an explicit, variable step size Runge
#   Kutta solver of order 4(5). It's also known as "Dormand-Prince".
#   Always try this solver first, as it is very efficient for a wide range of problems.
sol = solve_ivp(logistic_ode, t_span=[tstart, tstop], y0=N0, t_eval=tmesh)

# sol is a solution "Bunch" object with attributes listed in the solve_ivp documentation linked above.
time = sol.t # this gives you the times the solution was recorded at, which should be the same as tmesh if you specified t_eval as above.
N = sol.y[0,:] # our ODE "system" only had one equation, so the solution has shape (1, 1000 time points)

### Plot a solution set and either show it or return the plot object ###
plt.figure() # this creates a new figure object.
plt.plot(time, N, label='logistic eqn') # this plots the solution x against time, and gives it a label for the legend.
plt.xlabel('time') # this sets the label for the x-axis
plt.ylabel('population size') # this sets the label for the y-axis
plt.legend() # this adds a legend
plt.tight_layout() # this gives us less whitespace around the plot - good for saving the figure, but not necessary if you just want to show it.
plt.show() # this shows the plot.

Did you get a plot? If not, check that you have defined a function called logistic_ode with arguments t and then N, and that you have run the cell where you defined it.